In [44]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import normalize
from math import sqrt
from detorch import DE, Policy, Strategy
from detorch.config import default_config, Config
from typing import Type
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('..')
sys.path.append('viz')
from optimneuralucb import Network, DENeuralTSDiag, LenientDENeuralTSDiag
import viz_config
import logging

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = Config(default_config)
bounds = [-2.5, 1.5]
theta = torch.Tensor([0, 3, -2, -4, 1, 1]).to(device)
d = 1

%load_ext autoreload
%autoreload 2

In [45]:
def make_deterministic(seed=42):
    # PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Numpy
    np.random.seed(seed)

    # Built-in Python
    random.seed(seed)


In [46]:
def project_point(point):
    return torch.tensor([1, point, point ** 2, point ** 3, point ** 4, point ** 5]).to(device)


In [47]:
class PullPolicy(Policy):
    def __init__(self, eval_fn):
        super().__init__()
        self.point = torch.FloatTensor(1).uniform_(*bounds).to(device)
        self.params = nn.Parameter(
            self.point, requires_grad=False
        )
        self.eval_fn = eval_fn
        self.ucb = None

    def evaluate(self):
        self.transform()
        ucb, activation_grad, _, _ = self.eval_fn(self.point)
        self.activation_grad = activation_grad
        self.ucb = ucb
        # logging.info(self.point)
        # logging.info(ucb)
        return ucb

    def transform(self):
        self.point = torch.clip(self.params, *bounds).to(device)
        self.params = nn.Parameter(
            self.point, requires_grad=False
        )
        # return generate_feature_vector_from_point(self.point)


class DEConfig:
    n_step: int = 3
    population_size: int = 60
    differential_weight: float = 0.8
    crossover_probability: float = 0.9
    strategy: Strategy = Strategy.best1bin
    seed: int = 42


In [127]:
def reward_fn(point, add_noise=True):
    # random.seed(os.urandom(10))
    # seed = random.randint(0, 100000)
    # torch.manual_seed(seed)
    if add_noise:
        noise = torch.normal(0, 1, (1,)).item()
    else:
        noise = 0
    vec = project_point(point)
    value = theta @ vec.T
    return value + noise


def gen_warmup_vecs_and_rewards(n_warmup):
    vecs = torch.tensor([])
    rewards = torch.tensor([])
    for i in range(n_warmup):
        point = torch.FloatTensor(1).uniform_(bounds[0], bounds[1]).to(device)
        reward = torch.tensor([reward_fn(point)])

        vecs = torch.cat((vecs, point.cpu()))
        rewards = torch.cat((rewards, reward))
    vecs = vecs.view((n_warmup, -1))
    rewards = rewards.view((n_warmup, -1))
    return vecs, rewards


def plot_estimate(agent, trial, fn=None):
    n_points = 1000
    x = np.linspace(-2.5, 1.5, n_points)
    x_vec = []
    y = []
    for x_point in x:
        y.append(reward_fn(float(x_point), add_noise=False).cpu().numpy())

    y_pred = []
    cbs = []
    ucbs = []
    x_tns = torch.from_numpy(x)
    x_tns = x_tns.view(n_points, 1).float()

    for point in x_tns:
        sample, _, activ, cb = agent.get_sample(point.to(device))
        y_pred.append(activ)
        cbs.append(cb)
        ucbs.append(sample)

    y_pred = np.array(y_pred)
    cbs = np.array(cbs)

    point_played = agent.vec_history.squeeze(0).cpu().numpy()
    rewards_rec = agent.reward_history.squeeze(0).cpu().numpy()
    n_played = point_played.shape[0]

    # plt.ylim(-20, 10)
    # plt.ylim(-25, 10)
    plt.plot(x, y, color="tab:blue", label="Vraie fonction")
    plt.plot(x, y_pred, color="tab:orange", label="Estimation de la fonction")
    # plt.fill_between(x, y_pred, ucbs, color='tab:red', alpha=0.3)
    plt.fill_between(
        x, y_pred + cbs, y_pred - cbs, alpha=0.3, color="tab:orange", zorder=10
    )
    plt.plot(
        x,
        [0] * n_points,
        color="black",
        linestyle="dashed",
        label="Seuil bonne/mauvaise action",
    )
    plt.scatter(
        point_played[:n_played],
        rewards_rec[:n_played],
        color="black",
        alpha=0.5,
        label="Points joués précédemment",
    )
    plt.scatter(
        point_played[-1], rewards_rec[-1], color="green", label="Dernier point joué"
    )

    # title = rf"Estimation de la vraie fonction par NeuralTS ($\nu = {exploration_mult}$) après {trial} essais"
    # plt.title(title)
    
    plt.legend()
    if fn is None:
        filename = f"images/exp_poly/regTS_{n_trials}_trials_expl_{exploration_mult}_trial_{trial}.png"
    else:
        filename = f"images/exp_poly/{fn}.png"
    plt.savefig(filename)

    plt.clf()


def reseed_de(de_config):
    # Bypass default config of DE and modify seed so populations are different from invocation to invocation
    random.seed(os.urandom(100))
    de_config.seed = random.randint(0, 100000)


def find_best_member(eval_fn, de_config):
    reseed_de(de_config)
    config = Config(default_config)

    @config("policy")
    class PolicyConfig:
        policy: Type[Policy] = PullPolicy
        eval_fn: object = agent.get_sample

    config("de")(de_config)

    de = DE(config)
    de.train()

    return de.population[de.current_best]




In [76]:
run = 26
make_deterministic(run)

n_trials = 500
width = 100
net = Network(d, width).to(device)
reg = 1
dropout_rate = 0.1
exploration_mult = 10
delay = 0
reward_fn = reward_fn
de_config = DEConfig
de_policy = PullPolicy
style="ucb"
sampletype="f"

agent = DENeuralTSDiag(net, nu=exploration_mult, lamdba=reg, style=style, sampletype=sampletype)
# agent = LenientDENeuralTSDiag(net, nu=exploration_mult, lamdba=reg, reward_sample_thresholds=[float('-inf'), 0])


In [77]:
vecs, rewards = gen_warmup_vecs_and_rewards(10)
vecs, rewards = vecs.to(device), rewards.to(device)
example_vec = vecs[0]

In [ ]:
# Should we disable dropout when update U ?

### Warmup

In [86]:
for i in range(len(rewards)):
    agent.vec_history = vecs[:i + 1]
    agent.reward_history = rewards[:i + 1]

    if sampletype == "f":
        eval_mask = (
            torch.distributions.Bernoulli(
                torch.full_like(example_vec, (1 - dropout_rate))
            ).sample()
            / (1 - dropout_rate)
        ).to(device)

        training_mask = (
            torch.distributions.Bernoulli(
                torch.full_like(agent.vec_history, (1 - dropout_rate))
            ).sample()
            / (1 - dropout_rate)
        ).to(device)

        agent.net.set_mask(eval_mask)

    vec = vecs[i]
    activ, grad = agent.compute_activation_and_grad(vec)
    agent.U += grad * grad

    agent.net.set_mask(training_mask)
    
    agent.train(i + 1)

In [79]:
plot_estimate(agent, 0,  f'{style}_expl_{exploration_mult}_{sampletype}_run{run}_trial_0_prise2')

INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:

<Figure size 980x700 with 0 Axes>

### Train 1 run

In [75]:
for i in range(n_trials):
    if sampletype == "f":
        eval_mask = (
            torch.distributions.Bernoulli(
                torch.full_like(example_vec, (1 - dropout_rate))
            ).sample()
            / (1 - dropout_rate)
        ).to(device)


        agent.net.set_mask(eval_mask)
    best_member = find_best_member(agent.get_sample, de_config)
    best_member_grad = best_member.activation_grad
    a_t = best_member.point.unsqueeze(0)
    r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

    logging.info(f"trial: {i}")

    agent.U += best_member_grad * best_member_grad

    agent.vec_history = torch.cat((agent.vec_history, a_t))
    agent.reward_history = torch.cat((agent.reward_history, r_t))

    if sampletype == "f":
        training_mask = (
            torch.distributions.Bernoulli(
                torch.full_like(agent.vec_history, (1 - dropout_rate))
            ).sample()
            / (1 - dropout_rate)
        ).to(device)

        agent.net.set_mask(training_mask)

    n_steps = min(agent.reward_history.shape[0], 100)
    agent.train(n_steps)
    if (i + 1) % 100 == 0:
        plot_estimate(
            agent,
            0,
            f"{style}_expl_{exploration_mult}_{sampletype}_run{run}_trial_{i+1}_prise2",
        )


INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:No mask
INFO:root:

KeyboardInterrupt: Interrupted by user

In [88]:
def compute_jaccard(found_solution, true_solution):
    found_sol_list = found_solution.tolist()
    true_sol_list = true_solution.tolist()

    n_in_inter = 0

    for vec in found_sol_list:
        n_in_inter += vec in true_sol_list

    return n_in_inter / (len(found_solution) + len(true_solution) - n_in_inter), n_in_inter

In [89]:
x_arr = np.linspace(-2.5, 1.5, 1000)
x_arr = x_arr.reshape(1000, 1)
x = torch.from_numpy(x_arr).to(device).float()
y = []
for point in x:
    y.append(reward_fn(point, add_noise=False).cpu().numpy())

y = np.array(y)
true_sol = x[np.where(y >= 0)[0]].to(device).float()
n_true_sol = len(true_sol)
logging.info(n_true_sol)

INFO:root:419


In [92]:
metrics_dict = {}
algos = ["UCB", "regTS", "lenientTS"]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for exploration_mult in [1, 10]:
        metrics_dict[algo][str(exploration_mult)] = {}
        metrics_dict[algo][str(exploration_mult)]["jaccards"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_inter"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_found"] = []
        b = 0
    
        for i in range(b, 50):
            logging.info(f"at algo: {algo}, expl_mult: {exploration_mult}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 100
            width = 100
            net = Network(d, width).to(device)
            reg = 1
            delay = 0
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            lr = 1e-2

            if algo == "UCB":
                agent = DENeuralTSDiag(net, nu=exploration_mult, lamdba=reg, style="ucb")
            elif algo == "regTS":
                agent = DENeuralTSDiag(net, nu=exploration_mult, lamdba=reg, style="ts")
            elif algo == "lenientTS":
                agent = LenientDENeuralTSDiag(net, nu=exploration_mult, lamdba=reg, reward_sample_thresholds=[float('-inf'), 0])

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            for j in range(len(rewards)):
                agent.vec_history = vecs[:j + 1]
                agent.reward_history = rewards[:j + 1]
                vec = vecs[j]
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad
                agent.train(min(j + 1, max_n_steps))

            for j in range(n_trials):
                best_member = find_best_member(agent.get_sample, de_config=de_config)
                best_member_grad = best_member.activation_grad
                a_t = best_member.point.unsqueeze(0)
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                agent.U += best_member_grad * best_member_grad
                
                agent.vec_history = torch.cat((agent.vec_history, a_t))
                agent.reward_history = torch.cat((agent.reward_history, r_t))

                n_steps = min(agent.reward_history.shape[0], max_n_steps) 
                agent.train(n_steps, lr)

            # Stop using mask in evaluation
            agent.net.set_mask(None)
            sol = agent.find_solution_in_vecs(x, 0).to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(exploration_mult)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(exploration_mult)]["percent_inter"].append(percent_inter)
            metrics_dict[algo][str(exploration_mult)]["percent_found"].append(percent_found)

            if percent_found == 0:
                logging.info(f"Found no solution for run {i}")
                plot_estimate(agent, n_trials, fn=f"no_sol_{algo}_expl_{exploration_mult}_100_trials_seed_{i}.png")

            logging.info(f'jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}')

torch.save(metrics_dict, f'metrics_dict_exp_synth_{max_n_steps}.pth')


INFO:root:{}
INFO:root:at algo: UCB, expl_mult: 1, run: 0


KeyboardInterrupt: 

In [ ]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for expl_mult in metrics_dict[algo].keys():
        logging.info(f'Expl mult: {expl_mult}')
        for metric in  metrics_dict[algo][expl_mult].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][expl_mult][metric])} +- {np.std(metrics_dict[algo][expl_mult][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][expl_mult][metric])}, {np.max(metrics_dict[algo][expl_mult][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: regTS
INFO:root:Expl mult: 1
INFO:root:Metric: jaccards
INFO:root:mean: 0.4016826089008265
INFO:root:std: 0.0834754914683085
INFO:root:min: 0.0
INFO:root:max: 0.49403341288782815
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.9535538061313876
INFO:root:std: 0.14281898080679364
INFO:root:min: 0.0
INFO:root:max: 1.0
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.40782816229116947
INFO:root:std: 0.08747677971403428
INFO:root:min: 0.0
INFO:root:max: 0.5059665871121718
INFO:root:============================================
INFO:root:Expl mult: 10
INFO:root:Metric: jaccards
INFO:root:mean: 0.4261434857354548
INFO:root:std: 0.057963423621844215
INFO:root:min: 0.15035799522673032
INFO:root:max: 0.5115740740740741
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.7861245111975879
INFO:root:std: 0.14560173149385

## Test avec dropout plutot que bonus d'exploration

In [131]:
metrics_dict = {}
algos = ["regTS"]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for dropout_rate in [0.2, 0.5, 0.8]:
        metrics_dict[algo][str(dropout_rate)] = {}
        metrics_dict[algo][str(dropout_rate)]["jaccards"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_inter"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_found"] = []
        metrics_dict[algo][str(dropout_rate)]["fails"] = 0
        b = 0

        for i in range(b, 50):
            logging.info(f"at algo: {algo}, dropout: {dropout_rate}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 100
            width = 100
            net = Network(d, width).to(device)
            reg = 1
            sampletype = "f"
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            bern_p = 1 - dropout_rate
            p_vec = torch.tensor([bern_p] * width)

            if algo == "UCB":
                agent = DENeuralTSDiag(
                    net, lamdba=reg, style="ucb", sampletype=sampletype
                )
            elif algo == "regTS":
                agent = DENeuralTSDiag(
                    net, lamdba=reg, style="ts", sampletype=sampletype
                )
            elif algo == "lenientTS":
                agent = LenientDENeuralTSDiag(
                    net,
                    lamdba=reg,
                    reward_sample_thresholds=[float("-inf"), 0],
                    sampletype=sampletype,
                )

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)

            # Warmup
            for j in range(len(rewards)):
                agent.vec_history = vecs[: j + 1]
                agent.reward_history = rewards[: j + 1]

                if sampletype == "f":
                    eval_mask = (
                        torch.distributions.Bernoulli(
                            p_vec,
                        ).sample()
                        / bern_p  # Rescales the output by 1 - dropout, like the dropout layer of pytorch
                    ).to(device)

                    agent.net.set_mask(eval_mask)
                vec = vecs[j]
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad
                if sampletype == "f":
                    training_mask = (
                        torch.distributions.Bernoulli(
                            p_vec.repeat(agent.reward_history.shape[0], 1)
                        ).sample()
                        / bern_p  # Rescales the output by 1 - dropout, like the dropout layer of pytorch
                    ).to(device)

                    agent.net.set_mask(training_mask)

                agent.train(j + 1)

            # Train
            for j in range(n_trials):
                if sampletype == "f":
                    eval_mask = (
                        torch.distributions.Bernoulli(
                            p_vec,
                        ).sample()
                        / bern_p  # Rescales the output by 1 - dropout, like the dropout layer of pytorch
                    ).to(device)

                    agent.net.set_mask(eval_mask)

                best_member = find_best_member(agent.get_sample, de_config=de_config)
                best_member_grad = best_member.activation_grad
                a_t = best_member.point.unsqueeze(0)
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                if best_member_grad is None:
                    break
                agent.U += best_member_grad * best_member_grad

                agent.vec_history = torch.cat((agent.vec_history, a_t))
                agent.reward_history = torch.cat((agent.reward_history, r_t))

                if sampletype == "f":
                    training_mask = (
                        torch.distributions.Bernoulli(
                            p_vec.repeat(agent.reward_history.shape[0], 1)
                        ).sample()
                        / bern_p  # Rescales the output by 1 - dropout, like the dropout layer of pytorch
                    ).to(device)

                    agent.net.set_mask(training_mask)

                n_steps = min(agent.reward_history.shape[0], max_n_steps)
                agent.train(n_steps)

            if best_member_grad is None:
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1
                logging.info(f"Encountered a fail in {algo} {dropout_rate} because of nans")
                continue
            # Stop using mask in eval
            agent.net.set_mask(None)
            sol = agent.find_solution_in_vecs(x, 0).to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(dropout_rate)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(dropout_rate)]["percent_inter"].append(percent_inter)
            metrics_dict[algo][str(dropout_rate)]["percent_found"].append(percent_found)

            if percent_found == 0:
                logging.info(f"Found no solution for run {i}")
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1

                plot_estimate(
                    agent,
                    n_trials,
                    fn=f"no_sol_{algo}_drop_{dropout_rate}_100_trials_seed_{i}.png",
                )

            logging.info(
                f"jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}"
            )

torch.save(metrics_dict, f"metrics_dict_exp_synth_{max_n_steps}.pth")


INFO:root:{}
INFO:root:at algo: regTS, dropout: 0.2, run: 0
INFO:root:jaccard: 0.21689497716894976, percent_inter: 0.8333333333333334, percent_found: 0.22673031026252982
INFO:root:at algo: regTS, dropout: 0.2, run: 1
INFO:root:jaccard: 0.3201754385964912, percent_inter: 0.7978142076502732, percent_found: 0.34844868735083534
INFO:root:at algo: regTS, dropout: 0.2, run: 2
INFO:root:Found no solution for run 2
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 3
INFO:root:jaccard: 0.27906976744186046, percent_inter: 0.916030534351145, percent_found: 0.2863961813842482
INFO:root:at algo: regTS, dropout: 0.2, run: 4
INFO:root:jaccard: 0.3333333333333333, percent_inter: 0.7417840375586855, percent_found: 0.37708830548926014
INFO:root:at algo: regTS, dropout: 0.2, run: 5
INFO:root:Found no solution for run 5
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 6
INFO:root:jaccard: 0.180

<Figure size 980x700 with 0 Axes>

In [132]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for dropout_rate in metrics_dict[algo].keys():
        logging.info(f'dropout: {dropout_rate}')
        for metric in  metrics_dict[algo][dropout_rate].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][dropout_rate][metric])} +- {np.std(metrics_dict[algo][dropout_rate][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][dropout_rate][metric])}, {np.max(metrics_dict[algo][dropout_rate][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: regTS
INFO:root:dropout: 0.2
INFO:root:Metric: jaccards
INFO:root:mean: 0.15525933024671057 +- 0.14038302003011333 
INFO:root:interval:  [0.0, 0.36425339366515835]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.542565383738888 +- 0.3952446085885617 
INFO:root:interval:  [0.0, 1.0]
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.17178997613365155 +- 0.1558152646008552 
INFO:root:interval:  [0.0, 0.4081145584725537]
INFO:root:============================================
INFO:root:Metric: fails
INFO:root:mean: 16.0 +- 0.0 
INFO:root:interval:  [16, 16]
INFO:root:============================================
INFO:root:dropout: 0.5
INFO:root:Metric: jaccards
INFO:root:mean: 0.14418710798608372 +- 0.13288277581564642 
INFO:root:interval:  [0.0, 0.3986784140969163]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean